裁判主客场偏好

In [11]:
import pandas as pd

file_path = 'C:\\Users\\Hsiun\\Desktop\\COMP0036\\new_epl_training.csv' 

referee_data = pd.read_csv(file_path)
df = pd.DataFrame(referee_data)

# 计算裁判对主客场的判罚差异
df['Fouls_Difference'] = df['HF'] - df['AF']
df['Yellows_Difference'] = df['HY'] - df['AY']
df['Reds_Difference'] = df['HR'] - df['AR']

# 裁判偏好指数
df['Referee_Bias_Index'] = df[['Fouls_Difference', 'Yellows_Difference', 'Reds_Difference']].mean(axis=1)

df.to_csv('C:\\Users\\Hsiun\\Desktop\\COMP0036\\updated_new_epl_training.csv', index=False)

print("Updated CSV file with Referee Bias Index has been saved to the original location.")

Updated CSV file with Referee Bias Index has been saved to the original location.


赛后ELO评分

In [3]:
import pandas as pd

file_path = 'C:\\Users\\Hsiun\\Desktop\\COMP0036\\new_epl_training.csv'
df = pd.read_csv(file_path)

initial_elo = 1500
elo_ratings = {team: initial_elo for team in pd.concat([df['HomeTeam'], df['AwayTeam']]).unique()}

def update_elo(home_team, away_team, home_goals, away_goals, k=30):
    # 计算预期得分
    e_home = 1 / (1 + 10 ** ((elo_ratings[away_team] - elo_ratings[home_team]) / 400))
    e_away = 1 / (1 + 10 ** ((elo_ratings[home_team] - elo_ratings[away_team]) / 400))

    # 实际得分
    s_home, s_away = (0.5, 0.5) if home_goals == away_goals else ((1, 0) if home_goals > away_goals else (0, 1))

    # 更新Elo评分
    elo_ratings[home_team] += k * (s_home - e_home)
    elo_ratings[away_team] += k * (s_away - e_away)

# 用于存储每场比赛后的Elo评分
df['HomeTeam_Elo_After_Match'] = 0
df['AwayTeam_Elo_After_Match'] = 0

# 应用Elo评分更新，并记录每场比赛后的评分
for index, row in df.iterrows():
    update_elo(row['HomeTeam'], row['AwayTeam'], row['FTHG'], row['FTAG'])
    df.at[index, 'HomeTeam_Elo_After_Match'] = elo_ratings[row['HomeTeam']]
    df.at[index, 'AwayTeam_Elo_After_Match'] = elo_ratings[row['AwayTeam']]

new_file_path = 'C:\\Users\\Hsiun\\Desktop\\COMP0036\\updated_new_epl_training.csv'
df.to_csv(new_file_path, index=False)

print("CSV file with dynamic Elo ratings has been saved.")

CSV file with dynamic Elo ratings has been saved.


赛前ELO评分

In [12]:
import pandas as pd

file_path = 'C:\\Users\\Hsiun\\Desktop\\COMP0036\\new_epl_training.csv'
df = pd.read_csv(file_path)

initial_elo = 1500
elo_ratings = {team: initial_elo for team in pd.concat([df['HomeTeam'], df['AwayTeam']]).unique()}

def update_elo(home_team, away_team, home_goals, away_goals, k=30):
    # 计算预期得分
    e_home = 1 / (1 + 10 ** ((elo_ratings[away_team] - elo_ratings[home_team]) / 400))
    e_away = 1 / (1 + 10 ** ((elo_ratings[home_team] - elo_ratings[away_team]) / 400))

    # 实际得分
    s_home, s_away = (0.5, 0.5) if home_goals == away_goals else ((1, 0) if home_goals > away_goals else (0, 1))

    # 更新Elo评分
    new_home_elo = elo_ratings[home_team] + k * (s_home - e_home)
    new_away_elo = elo_ratings[away_team] + k * (s_away - e_away)

    return new_home_elo, new_away_elo

df['HomeTeam_Elo_Before_Match'] = 0
df['AwayTeam_Elo_Before_Match'] = 0

# 应用Elo评分更新，并记录每场比赛前的评分
for index, row in df.iterrows():
    # 记录比赛前的Elo评分
    df.at[index, 'HomeTeam_Elo_Before_Match'] = elo_ratings[row['HomeTeam']]
    df.at[index, 'AwayTeam_Elo_Before_Match'] = elo_ratings[row['AwayTeam']]

    # 更新Elo评分
    new_home_elo, new_away_elo = update_elo(row['HomeTeam'], row['AwayTeam'], row['FTHG'], row['FTAG'])
    elo_ratings[row['HomeTeam']] = new_home_elo
    elo_ratings[row['AwayTeam']] = new_away_elo

new_file_path = 'C:\\Users\\Hsiun\\Desktop\\COMP0036\\updated_new_epl_training.csv'
df.to_csv(new_file_path, index=False)

print("CSV file with pre-match Elo ratings has been saved.")

CSV file with pre-match Elo ratings has been saved.


进球转换率和射正比例

In [13]:
import pandas as pd

file_path = 'C:\\Users\\Hsiun\\Desktop\\COMP0036\\new_epl_training.csv'
df = pd.read_csv(file_path)

df['HomeTeam_Offensive_Shooting_Efficiency'] = 0.0
df['AwayTeam_Offensive_Shooting_Efficiency'] = 0.0
df['HomeTeam_Offensive_Shots_on_Target_Efficiency'] = 0.0
df['AwayTeam_Offensive_Shots_on_Target_Efficiency'] = 0.0

# 计算每场比赛的进攻和防守效率
for index, row in df.iterrows():
    # 进攻效率
    home_goals = row['FTHG']
    away_goals = row['FTAG']
    home_shots = row['HS']
    away_shots = row['AS']
    home_shots_on_target = row['HST']
    away_shots_on_target = row['AST']


    # 计算并更新进攻效率
    df.at[index, 'HomeTeam_Offensive_Shooting_Efficiency'] = home_goals / home_shots if home_shots > 0 else 0
    df.at[index, 'AwayTeam_Offensive_Shooting_Efficiency'] = away_goals / away_shots if away_shots > 0 else 0
    df.at[index, 'HomeTeam_Offensive_Shots_on_Target_Efficiency'] = home_goals / home_shots_on_target if home_shots_on_target > 0 else 0
    df.at[index, 'AwayTeam_Offensive_Shots_on_Target_Efficiency'] = away_goals / away_shots_on_target if away_shots_on_target > 0 else 0

new_file_path = 'C:\\Users\\Hsiun\\Desktop\\COMP0036\\updated_new_epl_training.csv'
df.to_csv(new_file_path, index=False)

print("CSV file with single match offensive efficiency has been saved.")


CSV file with single match offensive efficiency has been saved.


赛季进攻效率，防守效率

In [15]:
import pandas as pd

file_path = 'C:\\Users\\Hsiun\\Desktop\\COMP0036\\new_epl_training.csv'
df = pd.read_csv(file_path)

# 假设每个赛季有380场比赛
matches_per_season = 380

# 为DataFrame添加新列以存储每场比赛的效率
columns = ['HomeTeam_Season_Offensive_Shooting_Efficiency', 'AwayTeam_Season_Offensive_Shooting_Efficiency',
           'HomeTeam_Season_Offensive_Shots_on_Target_Efficiency', 'AwayTeam_Season_Offensive_Shots_on_Target_Efficiency',
           'HomeTeam_Season_Defensive_Shooting_Efficiency', 'AwayTeam_Season_Defensive_Shooting_Efficiency',
           'HomeTeam_Season_Defensive_Shots_on_Target_Efficiency', 'AwayTeam_Season_Defensive_Shots_on_Target_Efficiency']

for col in columns:
    df[col] = 0.0

# 遍历每个赛季
for i in range(0, len(df), matches_per_season):
    season_df = df.iloc[i:i + matches_per_season]

    # 统计数据
    season_team_data = {}
    for index, row in season_df.iterrows():
        for team, is_home in [(row['HomeTeam'], True), (row['AwayTeam'], False)]:
            if team not in season_team_data:
                season_team_data[team] = {'TotalShots': 0, 'TotalShotsOnTarget': 0, 'TotalGoalsScored': 0, 
                                          'TotalShotsConceded': 0, 'TotalShotsOnTargetConceded': 0, 'TotalGoalsConceded': 0}

            goals = row['FTHG'] if is_home else row['FTAG']
            shots = row['HS'] if is_home else row['AS']
            shots_on_target = row['HST'] if is_home else row['AST']
            goals_conceded = row['FTAG'] if is_home else row['FTHG']
            shots_conceded = row['AS'] if is_home else row['HS']
            shots_on_target_conceded = row['AST'] if is_home else row['HST']

            season_team_data[team]['TotalShots'] += shots
            season_team_data[team]['TotalShotsOnTarget'] += shots_on_target
            season_team_data[team]['TotalGoalsScored'] += goals
            season_team_data[team]['TotalShotsConceded'] += shots_conceded
            season_team_data[team]['TotalShotsOnTargetConceded'] += shots_on_target_conceded
            season_team_data[team]['TotalGoalsConceded'] += goals_conceded

    # 计算效率
    for index, row in season_df.iterrows():
        home_team = row['HomeTeam']
        away_team = row['AwayTeam']
        home_data = season_team_data[home_team]
        away_data = season_team_data[away_team]

        df.at[index, 'HomeTeam_Season_Offensive_Shooting_Efficiency'] = home_data['TotalGoalsScored'] / home_data['TotalShots'] if home_data['TotalShots'] > 0 else 0
        df.at[index, 'AwayTeam_Season_Offensive_Shooting_Efficiency'] = away_data['TotalGoalsScored'] / away_data['TotalShots'] if away_data['TotalShots'] > 0 else 0
        df.at[index, 'HomeTeam_Season_Offensive_Shots_on_Target_Efficiency'] = home_data['TotalGoalsScored'] / home_data['TotalShotsOnTarget'] if home_data['TotalShotsOnTarget'] > 0 else 0
        df.at[index, 'AwayTeam_Season_Offensive_Shots_on_Target_Efficiency'] = away_data['TotalGoalsScored'] / away_data['TotalShotsOnTarget'] if away_data['TotalShotsOnTarget'] > 0 else 0
        df.at[index, 'HomeTeam_Season_Defensive_Shooting_Efficiency'] = home_data['TotalGoalsConceded'] / home_data['TotalShotsConceded'] if home_data['TotalShotsConceded'] > 0 else 0
        df.at[index, 'AwayTeam_Season_Defensive_Shooting_Efficiency'] = away_data['TotalGoalsConceded'] / away_data['TotalShotsConceded'] if away_data['TotalShotsConceded'] > 0 else 0
        df.at[index, 'HomeTeam_Season_Defensive_Shots_on_Target_Efficiency'] = home_data['TotalGoalsConceded'] / home_data['TotalShotsOnTargetConceded'] if home_data['TotalShotsOnTargetConceded'] > 0 else 0
        df.at[index, 'AwayTeam_Season_Defensive_Shots_on_Target_Efficiency'] = away_data['TotalGoalsConceded'] / away_data['TotalShotsOnTargetConceded'] if away_data['TotalShotsOnTargetConceded'] > 0 else 0


new_file_path = 'C:\\Users\\Hsiun\\Desktop\\COMP0036\\updated_new_epl_training.csv'
df.to_csv(new_file_path, index=False)

print("CSV file with updated offensive and defensive efficiency for each match has been saved.")


CSV file with updated offensive and defensive efficiency for each match has been saved.


主队优势和客队表现

In [16]:
import pandas as pd

file_path = 'C:\\Users\\Hsiun\\Desktop\\COMP0036\\new_epl_training.csv'
df = pd.read_csv(file_path)

# 假设每个赛季有380场比赛
matches_per_season = 380


df['HomeTeam_HomeWinRate'] = 0.0
df['AwayTeam_AwayWinRate'] = 0.0
df['HomeTeam_HomeGoalsRatio'] = 0.0
df['AwayTeam_AwayGoalsRatio'] = 0.0
df['HomeTeam_HomeWinsRatio'] = 0.0
df['AwayTeam_AwayWinsRatio'] = 0.0

# 遍历每个赛季
for i in range(0, len(df), matches_per_season):
    season_df = df.iloc[i:i + matches_per_season]

    # 统计数据
    season_team_data = {}
    for index, row in season_df.iterrows():
        for team, is_home in [(row['HomeTeam'], True), (row['AwayTeam'], False)]:
            if team not in season_team_data:
                season_team_data[team] = {'TotalGoals': 0, 'TotalWins': 0, 'HomeGoals': 0, 'HomeWins': 0, 'AwayGoals': 0, 'AwayWins': 0}
            
            goals = row['FTHG'] if is_home else row['FTAG']
            win = (row['FTR'] == 'H' and is_home) or (row['FTR'] == 'A' and not is_home)

            season_team_data[team]['TotalGoals'] += goals
            if win:
                season_team_data[team]['TotalWins'] += 1
            
            if is_home:
                season_team_data[team]['HomeGoals'] += goals
                if win:
                    season_team_data[team]['HomeWins'] += 1
            else:
                season_team_data[team]['AwayGoals'] += goals
                if win:
                    season_team_data[team]['AwayWins'] += 1

    # 计算比率
    for index, row in season_df.iterrows():
        home_team = row['HomeTeam']
        away_team = row['AwayTeam']
        home_data = season_team_data[home_team]
        away_data = season_team_data[away_team]

        df.at[index, 'HomeTeam_HomeWinRate'] = home_data['HomeWins'] / len(season_df[season_df['HomeTeam'] == home_team]) if len(season_df[season_df['HomeTeam'] == home_team]) > 0 else 0
        df.at[index, 'AwayTeam_AwayWinRate'] = away_data['AwayWins'] / len(season_df[season_df['AwayTeam'] == away_team]) if len(season_df[season_df['AwayTeam'] == away_team]) > 0 else 0

        df.at[index, 'HomeTeam_HomeGoalsRatio'] = home_data['HomeGoals'] / home_data['TotalGoals'] if home_data['TotalGoals'] > 0 else 0
        df.at[index, 'AwayTeam_AwayGoalsRatio'] = away_data['AwayGoals'] / away_data['TotalGoals'] if away_data['TotalGoals'] > 0 else 0

        df.at[index, 'HomeTeam_HomeWinsRatio'] = home_data['HomeWins'] / home_data['TotalWins'] if home_data['TotalWins'] > 0 else 0
        df.at[index, 'AwayTeam_AwayWinsRatio'] = away_data['AwayWins'] / away_data['TotalWins'] if away_data['TotalWins'] > 0 else 0

new_file_path = 'C:\\Users\\Hsiun\\Desktop\\COMP0036\\updated_new_epl_training.csv'
df.to_csv(new_file_path, index=False)

print("CSV file with new performance metrics has been saved.")


CSV file with new performance metrics has been saved.


处理英超数据

In [17]:
import pandas as pd

file_path = 'C:\\Users\\Hsiun\\Desktop\\COMP0036\\new_epl_training.csv' 

df = pd.read_csv(file_path)
df['EPL'] = 1

output_file_path = 'C:\\Users\\Hsiun\\Desktop\\COMP0036\\updated_new_epl_training.csv'  
df.to_csv(output_file_path, index=False)

print("Updated CSV file with EPL feature has been saved.")


Updated CSV file with EPL feature has been saved.


添加杯赛数据

In [18]:
import pandas as pd

seasons_file_path = 'C:\\Users\\Hsiun\\Desktop\\COMP0036\\new_epl_training.csv'  
cup_matches_file_path = 'C:\\Users\\Hsiun\\Desktop\\COMP0036\\External_Cups.csv'

seasons_df = pd.read_csv(seasons_file_path)
cup_matches_df = pd.read_csv(cup_matches_file_path)


# 对于赛季数据中存在而杯赛数据中不存在的列，用0填充
combined_df = pd.concat([seasons_df, cup_matches_df.reindex(columns=seasons_df.columns, fill_value=0)], ignore_index=True)

combined_df['Date'] = pd.to_datetime(combined_df['Date'])
combined_df.sort_values(by='Date', inplace=True)

output_file_path = 'C:\\Users\\Hsiun\\Desktop\\COMP0036\\combined_matches.csv'
combined_df.to_csv(output_file_path, index=False)

print("Combined CSV file with season and cup matches has been saved and sorted by date.")


Combined CSV file with season and cup matches has been saved and sorted by date.


杯赛因素

In [19]:
import pandas as pd
from datetime import datetime, timedelta


file_path = 'C:\\Users\\Hsiun\\Desktop\\COMP0036\\new_epl_training.csv'

df = pd.read_csv(file_path)

df['Date'] = pd.to_datetime(df['Date'], format='%Y/%m/%d')

df['HomeTeam_CupGamesLastMonth'] = 0
df['AwayTeam_CupGamesLastMonth'] = 0

# 为每个队伍分别计算过去一个月的杯赛比赛次数
for index, row in df.iterrows():
    # 计算当前日期的前一个月
    date_one_month_ago = row['Date'] - timedelta(days=30)
    
    # 过滤出过去一个月的杯赛比赛
    last_month_cup_games = df[(df['Date'] >= date_one_month_ago) & (df['Date'] < row['Date']) & (df['EPL'] == 0)]
    
    # 计算主队过去一个月的杯赛比赛次数
    home_team_cup_games = last_month_cup_games[(last_month_cup_games['HomeTeam'] == row['HomeTeam']) | (last_month_cup_games['AwayTeam'] == row['HomeTeam'])]
    df.at[index, 'HomeTeam_CupGamesLastMonth'] = len(home_team_cup_games)
    
    # 计算客队过去一个月的杯赛比赛次数
    away_team_cup_games = last_month_cup_games[(last_month_cup_games['HomeTeam'] == row['AwayTeam']) | (last_month_cup_games['AwayTeam'] == row['AwayTeam'])]
    df.at[index, 'AwayTeam_CupGamesLastMonth'] = len(away_team_cup_games)

df.to_csv('C:\\Users\\Hsiun\\Desktop\\COMP0036\\updated_new_epl_training.csv', index=False)
print("CSV file with new features has been saved.")

CSV file with new features has been saved.


团队士气+诅咒参数

In [20]:
import pandas as pd

file_path = 'C:\\Users\\Hsiun\\Desktop\\COMP0036\\new_epl_training.csv'  
df = pd.read_csv(file_path)

# 确保日期列以日期时间格式存储
df['Date'] = pd.to_datetime(df['Date'])
df.sort_values(by='Date', inplace=True)

def calculate_team_morale(df, team, current_date):
    # 获取该队伍过去五场比赛
    past_matches = df[((df['HomeTeam'] == team) | (df['AwayTeam'] == team)) & (df['Date'] < current_date)].tail(5)
    
    # 计算士气
    morale = 0
    for _, row in past_matches.iterrows():
        if row['HomeTeam'] == team:  # 如果球队是主队
            if row['FTHG'] > row['FTAG']:  # 主队赢
                morale += 3
            elif row['FTHG'] == row['FTAG']:  # 主队平
                morale -= 1
            else:  # 主队输
                morale -= 5
        else:  # 如果球队是客队
            if row['FTAG'] > row['FTHG']:  # 客队赢
                morale += 5
            elif row['FTAG'] == row['FTHG']:  # 客队平
                morale += 1
            else:  # 客队输
                morale -= 3
    return morale

def calculate_curse_parameter(df, home_team, away_team, current_date):
    # 获取两队过去五次交手的比赛
    past_matches = df[((df['HomeTeam'] == home_team) & (df['AwayTeam'] == away_team) | 
                      (df['HomeTeam'] == away_team) & (df['AwayTeam'] == home_team)) & 
                      (df['Date'] < current_date)].tail(5)
    
    # 计算诅咒参数
    curse_param = 0
    for _, row in past_matches.iterrows():
        if (row['HomeTeam'] == home_team and row['FTHG'] > row['FTAG']) or \
           (row['AwayTeam'] == home_team and row['FTAG'] > row['FTHG']):
            curse_param += 1
        elif (row['HomeTeam'] == away_team and row['FTHG'] > row['FTAG']) or \
             (row['AwayTeam'] == away_team and row['FTAG'] > row['FTHG']):
            curse_param -= 1
    return curse_param

df['HomeTeam_Morale'] = df.apply(lambda row: calculate_team_morale(df, row['HomeTeam'], row['Date']), axis=1)
df['AwayTeam_Morale'] = df.apply(lambda row: calculate_team_morale(df, row['AwayTeam'], row['Date']), axis=1)
df['Curse_Parameter'] = df.apply(lambda row: calculate_curse_parameter(df, row['HomeTeam'], row['AwayTeam'], row['Date']), axis=1)

output_path = 'C:\\Users\\Hsiun\\Desktop\\COMP0036\\updated_new_epl_training.csv'
df.to_csv(output_path, index=False)

print("CSV file with new features has been saved.")


CSV file with new features has been saved.
